In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.optimize import least_squares
%matplotlib inline

from heston import UnivariateHestonSV, MultivariateHestonSV
from heston import make_surface
from heston import MultiAssetsOption, MultiAssetsAsianPut, MultiAssetsAsianCall
from heston import MultiAssetsDiscreteKIEuropeanPut, MultiAssetsAsianZeroCostCollar
from heston import MultiAssetsBestOfAsianCall, MultiAssetsWorstOfDiscreteKIEuropeanPut
from heston import structure_constructor

In [2]:
data = pd.read_excel('data/equities.xlsx', index_col=0)
assets = data.corp.unique()
data.head()

,corp,strike,mat,iv,S0,adv,tot_shr,owned_shr,avg_buy_px,bs_px
0,Green Co.,30,0.083333,NaN,34,217000,30,4.25,31.5,NaN
1,Green Co.,30,0.250000,0.3747,34,217000,30,4.25,31.5,5.143945
2,Green Co.,30,0.500000,0.3686,34,217000,30,4.25,31.5,6.126322
3,Green Co.,35,0.083333,0.4211,34,217000,30,4.25,31.5,1.270197
4,Green Co.,35,0.250000,0.3418,34,217000,30,4.25,31.5,2.044304


## Calibration

In [3]:
# Use this script if you want to re-calibrate multivariate heston model
assets = data.corp.unique()
mv_hes = MultivariateHestonSV(assets)
fit_res = mv_hes.calibrate(
    data, px=[34,15,25], r=[0.045]*3,
    cov_s=np.array([
        [1, 0.3, 0.3],
        [0.3, 1, 0.3],
        [0.3, 0.3, 1]
    ])
)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.6504e+00                                    2.35e+02    
       1              2         3.6815e-01      1.28e+00       3.80e-01       2.28e+01    
       2              3         1.1837e-01      2.50e-01       4.33e-01       3.68e-01    
       3              4         5.7879e-02      6.05e-02       3.17e-01       5.96e-01    
       4              5         4.2593e-02      1.53e-02       1.13e-01       7.41e-02    
       5              6         3.9224e-02      3.37e-03       2.19e-01       5.60e-01    
       6              7         2.8398e-02      1.08e-02       3.53e+00       4.82e-02    
       7              8         2.4080e-02      4.32e-03       2.64e+00       1.48e-02    
       8              9         1.9916e-02      4.16e-03       2.59e+00       3.43e-02    
       9             10         1.7406e-02      2.51e-03       2.98e+00       1.82e-02    

## Load Pre-Calibrated Model

In [3]:
assets = data.corp.unique()
mv_hes = MultivariateHestonSV(assets)
mv_hes.set_params(
    {
        # parameters list: ['kappa', 'v0', 'theta', 'eta', 'rho_sv']
        assets[0]: [12.22153077,  0.19999999,  0.10679624,  1.11942453, -0.79999979],
        assets[1]: [ 0.69115605,  0.11214887,  0.4196924 ,  0.19383575, -0.79890507],
        assets[2]: [ 2.0236936 ,  0.14899818,  0.30447942,  0.4452153 , -0.79999991],
        # risk free rate
        'r': 0.045,
        # stock covariance matrix
        'cov_s': np.array([
            [1, 0.3, 0.3],
            [0.3, 1, 0.3],
            [0.3, 0.3, 1]
        ])
    }
)

## Valuations of 0-Cost-Collar (w/ Different Ks)

In [7]:
s_0 = np.array([34, 15, 25]) # time-0 stock price
re_eval = False # change to True if you want to evaluate everything again.

# client input, different put strikes
# (i.e. the payoff the client wants to lock-in upon maturity)
put_strike_terms = [
    np.array([32, 13, 24]),
    np.array([30, 12, 22]),
    np.array([28, 11, 20])
]

if not re_eval:
    call_strike_terms = [
        np.array([39.572, 20.113, 29.174]),
        np.array([42.705, 21.594, 32.053]),
        np.array([45.266, 24.261, 35.242])
    ]
else:
    call_strike_terms = [None, None, None]

collars = []
for i, ks in enumerate(put_strike_terms):
    # call strikes are solved by least squares
    print(f'------------------------ Structure {i} ------------------------')
    zero_collar = MultiAssetsAsianZeroCostCollar(
        mv_hes, spots=s_0, put_Ks=ks, call_Ks=call_strike_terms[i], T=2)
    collars.append(zero_collar)

------------------------ Structure 0 ------------------------
------------------------ Structure 1 ------------------------
------------------------ Structure 2 ------------------------


In [6]:
# view the structure
import time
for i, structure in enumerate(collars):
    print(f'------------------------ Structure {i} ------------------------')
    print(repr(structure), '\n')
    px, se = structure.mc_price(spots=s_0, n_paths=3000)
    delta = structure.mc_delta(spots=s_0, n_paths=3000)
    print('Structure Price:', px)
    print('(SE):', se)
    print('Structure Delta:', delta, '\n')

------------------------ Structure 0 ------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=32.000, 13.000, 24.000, T=2
- MultiAssetsAsianCall: K=39.572, 20.113, 29.174, T=2 

Simulating paths...


100% (3000 of 3000) |#####################| Elapsed Time: 0:00:07 Time: 0:00:07


Simulating paths...


100% (3000 of 3000) |#####################| Elapsed Time: 0:00:15 Time: 0:00:15


Structure Price: [-0.00820493  0.0236197  -0.32928955]
(SE): [0.1278145  0.06078583 0.15767932]
Structure Delta: [-0.72930874 -0.55808233 -0.84223223]
------------------------ Structure 1 ------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=30.000, 12.000, 22.000, T=2
- MultiAssetsAsianCall: K=42.705, 21.594, 32.053, T=2 

Simulating paths...


...

Structure Price: [ 0.10983103  0.00068597 -0.2170945 ]
(SE): [0.10006279 0.05069604 0.1253033 ]
Structure Delta: [-0.54920707 -0.42675758 -0.65201799]
------------------------ Structure 2 ------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=28.000, 11.000, 20.000, T=2
- MultiAssetsAsianCall: K=45.266, 24.261, 35.242, T=2 

Simulating paths...


100% (3000 of 3000) |#####################| Elapsed Time: 0:00:07 Time: 0:00:07
  0% (19 of 3000) |                       | Elapsed Time: 0:00:00 ETA:  0:00:15

Simulating paths...


100% (3000 of 3000) |#####################| Elapsed Time: 0:00:14 Time: 0:00:14


Structure Price: [0.09595555 0.04868959 0.13340748]
(SE): [0.07746059 0.03564338 0.08870669]
Structure Delta: [-0.38366837 -0.28654886 -0.47684096]


## Valuation of Best-of-Call

In [5]:
boc = MultiAssetsBestOfAsianCall(
        mv_hes, collars[0], 
        T_entry=1, 
        T_mature=2, 
        S_init=s_0, 
        required_return=0.045)

woKI = MultiAssetsWorstOfDiscreteKIEuropeanPut(
        mv_hes, collars[0], boc, s_0,
        T_entry=1, 
        T_mature=2, 
        n_paths=3000,
    )

Solving barriers...
Simulating paths...


100% (3000 of 3000) |#####################| Elapsed Time: 0:00:48 Time: 0:00:48


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.3107e+00                                    1.33e+01    
       1              2         4.7308e-02      2.26e+00       5.44e+00       1.34e+00    
       2              3         7.4084e-05      4.72e-02       1.08e+00       1.26e-02    
       3              4         2.9746e-06      7.11e-05       4.63e-02       9.77e-03    
       4              5         5.1895e-07      2.46e-06       9.27e-03       1.04e-03    
       5              7         2.9203e-07      2.27e-07       9.74e-04       7.82e-04    
       6              8         3.2567e-08      2.59e-07       1.95e-03       2.61e-04    
       7             11         1.8355e-08      1.42e-08       2.43e-04       1.96e-04    
       8             13         1.2766e-08      5.59e-09       1.22e-04       1.64e-04    
       9             15         1.0351e-08      2.41e-09       6.08e-05       1.69e+00    

In [6]:
woKI

MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[28.199, 11.099, 19.111], H=[23.199, 6.099, 14.111], T_entry=1, T_mature=2

## The Entire Structure - An Example

In [ ]:
s_0 = np.array([34, 15, 25]) # time-0 stock price
# use pre-computed paths to speed things up
n = 5000
paths = mv_hes.simulate_paths_cfb(
    n_paths=n, n_nodes=252, T=2, dS=1e-4, S0=s_0)

 58% (2914 of 5000) |############         | Elapsed Time: 0:02:16 ETA:  0:01:16

In [13]:
SSc, SSf, SSb, XXc, XXf, XXb, VV = paths
paths_c = (SSc, XXc, VV)
paths_f = (SSf, XXf, VV)
paths_b = (SSb, XXb, VV)
disc_factor = np.exp(-0.045*2)

In [8]:
s_0 = np.array([34, 15, 25])
zero_collar = MultiAssetsAsianZeroCostCollar(
    mv_hes, spots=s_0, 
    put_Ks=np.array([32, 14, 23.5]), T=2,
    n_paths=5000, n_nodes_per_year=252,
    pre_computed_paths=paths_c
)
best_of_call = MultiAssetsBestOfAsianCall(
    mv_hes, zero_collar, 
    T_entry=1, T_mature=2, 
    S_init=s_0, required_return=0.045
)
worst_of_KIput = MultiAssetsWorstOfDiscreteKIEuropeanPut(
    mv_hes, zero_collar, best_of_call, s_0, 
    T_entry=1, T_mature=2, Ds=np.array([5]*3), n_paths=5000,
    pre_computed_paths=paths_c
)

structure = {
    'Zero Collar': zero_collar,
    'Best of Call': best_of_call,
    'worst of KI Put': worst_of_KIput
}

Solving call strikes...
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         4.7199e+00                                    1.12e+00    
       1              2         1.3711e-01      4.58e+00       6.56e+00       1.29e-01    
       2              3         4.2874e-04      1.37e-01       1.67e+00       6.98e-03    
       3              4         6.1503e-09      4.29e-04       1.05e-01       2.90e-05    
       4              5         3.1923e-25      6.15e-09       4.03e-04       7.07e-12    
`gtol` termination condition is satisfied.
Function evaluations 5, initial cost 4.7199e+00, final cost 3.1923e-25, first-order optimality 7.07e-12.
Solving barriers...
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         2.3908e+00                                    1.25e+01    
       1              2         4.6501e-02      2.34e+00       5.53e+00       1.

In [15]:
for k, obj in structure.items():
    print(repr(obj), '\n')

Zero Cost Collar:
+ MultiAssetsAsianPut: K=[32.000, 14.000, 23.500], T=2
- MultiAssetsAsianCall: K=[40.000, 17.924, 30.046], T=2 

MultiAssetsBestOfAsianCall: K=[40.000, 17.924, 30.046], R_required=0.045, T_entry=1, T_mature=2 

MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[28.071, 11.663, 19.120], H=[23.071, 6.663, 14.120], T_entry=1, T_mature=2 



## Sanity Check: Pricing of the structure (price $\approx$ 0)

In [16]:
print('------------------------------------------')
for k, obj in structure.items():
    print(repr(obj))
    px, se = obj.mc_price(spots=s_0, pre_computed_paths=paths_c)
    delta = obj.mc_delta(spots=s_0, dS=1e-4, pre_computed_paths=paths)
    print('Price:', px)
    print('(SE)', se)
    print('Delta', delta)
    print('------------------------------------------')



------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[32.000, 14.000, 23.500], T=2
- MultiAssetsAsianCall: K=[40.000, 17.924, 30.046], T=2
Simulating paths...
Simulating paths...
Price: [2.02738673e-15 2.41734372e-15 3.92781827e-15]
(SE) [0.21442489 0.13476363 0.23934694]
Delta [[-0.65145169  0.          0.        ]
 [ 0.         -0.69507002  0.        ]
 [ 0.          0.         -0.72045429]]
------------------------------------------
MultiAssetsBestOfAsianCall: K=[40.000, 17.924, 30.046], R_required=0.045, T_entry=1, T_mature=2
Simulating paths...
Simulating paths...
Price: 3.864674806322516
(SE) 0.20116967767273844
Delta [[0.21286972 0.24397859 0.29126505]]
------------------------------------------
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[28.071, 11.663, 19.120], H=[23.071, 6.663, 14.120], T_entry=1, T_mature=2
Simulating paths...
Simulating paths...
Price: 3.864674806322516
(SE) 0.16206886689421005
Delta [[-0.0701974  -0.06423913 -0.08092108]]
-

## Optimized Structure (Pre computed paths)

In [14]:
s_0 = np.array([34, 15, 25])
adv = data.adv.unique() / 1e6
markup = (np.array([0.1, 0.1, 0.1]), 0.0)

def metric(K, D, R):
    x = -(100/np.sum(adv.dot(K)))*adv*np.exp(0.045*2)
    loss, structure, pct_adv, cap, deltas, slip = structure_constructor(
        model=mv_hes, x=x, put_Ks=K, Ds=D, R_req=R, adv=adv, 
        adv_thres=0.3, S0=s_0, cap_required=100, alpha=0.1, pre_computed_paths=paths, 
        premiums=markup)
    delta_collar, delta_boc, delta_KI = deltas
    delta_tot = delta_collar-delta_boc+delta_KI
    # print(structure)
    print('Put Strikes:', K)
    print('Shares:', x)
    print('Loss:', loss)
    print('|X/ADV|:', np.abs(pct_adv))
    print('Price Slippage', slip)
    print('Pct Price Slippage (BPs)', (slip/s_0)*1e4)
    print('Dollar Slippage', slip.dot(delta_tot.reshape(3,))[0])
    print('Dollar Premium', -markup[0].dot(x)+markup[1])
    print('Locked-in Capital:', cap)
    print('Deltas/Sell x Collar:', delta_collar)
    print('Deltas/Sell 1 Best of Call:', -delta_boc)
    print('Deltas/Buy 1 KI:', delta_KI)
    print('Deltas/Entire Structure:', delta_tot)
    return structure

In [15]:
structure = metric(np.array([26, 10.85, 17.5]), np.array([2, 2, 1.8]), R=0.045)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

Put Strikes: [26.   10.85 17.5 ]
Shares: [-0.90295229 -2.53825298 -3.33717851]
Loss: 0.30917940305287855
|X/ADV|: [[0.41327206 0.52813261 0.47526224]]
Price Slippage [[0.7676205  0.35989321 0.60599066]]
Pct Price Slippage (BPs) [[225.77073471 239.92880879 242.39626596]]
Dollar Slippage 0.4157633498893614
Dollar Premium 0.6778383784890145
Locked-in Capital: 100.0
Deltas/Sell x Collar: [[0.23282198 0.72578314 0.96747014]]
Deltas/Sell 1 Best of Call: [[-0.09723002 -0.32163365 -0.49666056]]
Deltas/Buy 1 KI: [[-0.04591192 -0.08198859 -0.08964926]]
Deltas/Entire Structure: [[0.08968004 0.32216089 0.38116032]]
----------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[26.000, 10.850, 17.500], T=2
- MultiAssetsAsianCall: K=[48.805, 24.689, 39.516], T=2
MultiAssetsBestOfAsianCall: K=[48.805, 24.689, 39.516], R_required=0.045, T_entry=1, T_mature=2
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[23.990, 7.680, 11.225], H=[21.990, 5.680, 9.425], T_entry=1, T_mature=2


In [11]:
markup = (np.array([0.15, 0.15, 0.15]), 0.0)
structure = metric(np.array([27.4, 11.65, 19.5]), np.array([2, 2, 1.8]), R=0.045)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

Put Strikes: [27.4  11.65 19.5 ]
Shares: [-0.82755337 -2.3263021  -3.05851521]
Loss: 3.452481164534325
|X/ADV|: [[8.19703647 3.72978456 0.69522658]]
Price Slippage [[6.40939792 1.39544714 0.91000718]]
Pct Price Slippage (BPs) [[1885.11703451  930.298093    364.0028721 ]]
Dollar Slippage -7.718489105954936
Dollar Premium 0.9318556016052537
Locked-in Capital: 100.00000000000001
Deltas/Sell x Collar: [[0.30231821 0.9089059  1.306043  ]]
Deltas/Sell 1 Best of Call: [[-0.11454012 -0.43098769 -0.6015973 ]]
Deltas/Buy 1 KI: [[-1.966535    1.79725037 -0.14687399]]
Deltas/Entire Structure: [[-1.77875691  2.27516858  0.55757171]]
----------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[27.400, 11.650, 19.500], T=2
- MultiAssetsAsianCall: K=[44.612, 21.213, 35.073], T=2
MultiAssetsBestOfAsianCall: K=[44.612, 21.213, 35.073], R_required=0.045, T_entry=1, T_mature=2
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[25.835, 9.138, 13.382], H=[23.835, 7.138, 11.582], T_ent

In [48]:
markup = (np.array([0.15, 0.15, 0.15]), 0.0)
structure = metric(np.array([27.4, 11.65, 19.5]), np.array([2, 2, 1.8]), R=0.045)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

Put Strikes: [27.4  11.65 19.5 ]
Shares: [-0.82755337 -2.3263021  -3.05851521]
Loss: 0.5472321891779234
|X/ADV|: [[0.6148653  0.61794756 0.61501152]]
Price Slippage [[1.86510002 0.42487161 0.81242816]]
Pct Price Slippage (BPs) [[548.55882859 283.24774251 324.97126476]]
Dollar Slippage 0.8097283672770561
Dollar Premium 0.9318556016052537
Locked-in Capital: 100.00000000000001
Deltas/Sell x Collar: [[0.29785418 0.91857618 1.29274094]]
Deltas/Sell 1 Best of Call: [[-0.11187334 -0.42037315 -0.64496774]]
Deltas/Buy 1 KI: [[-0.05255507 -0.12125501 -0.15453395]]
Deltas/Entire Structure: [[0.13342577 0.37694801 0.49323924]]
----------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[27.400, 11.650, 19.500], T=2
- MultiAssetsAsianCall: K=[44.644, 20.835, 35.223], T=2
MultiAssetsBestOfAsianCall: K=[44.644, 20.835, 35.223], R_required=0.045, T_entry=1, T_mature=2
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[25.950, 9.255, 13.906], H=[23.950, 7.255, 12.106], T_entry=1,

In [77]:
markup = (np.array([0.0, 0.0, 0.0]), 1.0)
structure = metric(np.array([31.5, 11, 22]), np.array([1.8, 2, 1.8]), R=0.05)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

Put Strikes: [31.5 11.  22. ]
Shares: [-0.76126844 -2.13997119 -2.81353589]
Loss: 1.0438732730035158
|X/ADV|: [[1.14946523 0.33914327 0.90543211]]
Price Slippage [[3.0634622  0.14907626 1.12630101]]
Pct Price Slippage (BPs) [[901.01829404  99.38417488 450.52040521]]
Dollar Slippage 1.6128428600033233
Dollar Premium 1.0
Locked-in Capital: 99.99999999999999
Deltas/Sell x Collar: [[0.47303023 0.59824648 1.67777114]]
Deltas/Sell 1 Best of Call: [[-0.1525642  -0.26958288 -0.74132532]]
Deltas/Buy 1 KI: [[-0.07103207 -0.12178621 -0.21028926]]
Deltas/Entire Structure: [[0.24943395 0.20687739 0.72615655]]
----------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[31.500, 11.000, 22.000], T=2
- MultiAssetsAsianCall: K=[40.258, 23.383, 31.903], T=2
MultiAssetsBestOfAsianCall: K=[40.258, 23.383, 31.903], R_required=0.05, T_entry=1, T_mature=2
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[30.139, 9.554, 17.239], H=[28.339, 7.554, 15.439], T_entry=1, T_mature=2


In [41]:
1e4*np.array([1.34994984,0.33257318,0.51510591])/np.array([34, 15, 25])

array([397.04407059, 221.71545333, 206.042364  ])

In [19]:
def slippage_sq_root(vol, x, adv, s_0):
    pct_px_slippage = 0.15 * vol * np.sqrt(
        (np.abs(x) / adv - 0.3).clip(0))
    px_slippage = pct_px_slippage * s_0
    return px_slippage

slippage_sq_root(np.sqrt(mv_hes.v0_vec), 
                 np.array([0.11181991 ,0.31767633 ,0.42480413]), 
                 adv,
                 np.array([34,15,25])
                ).dot(np.array([0.11181991 ,0.31767633 ,0.42480413]))

array([0.5796241])

In [16]:
SSc, SSf, SSb, XXc, XXf, XXb, VV = paths
paths_c = (SSc, XXc, VV)
direction = [-1, -1, 1]
print('------------------------------------------')
for i, (k, obj) in enumerate(structure.items()):
    print(repr(obj))
    px, se = obj.mc_price(spots=s_0, pre_computed_paths=paths_c)
    delta = obj.mc_delta(spots=s_0, dS=1e-4, pre_computed_paths=paths)
    print('Price:', px*direction[i])
    print('(SE)', se)
    print('Delta', delta*direction[i])
    print('------------------------------------------')

------------------------------------------
Zero Cost Collar:
+ MultiAssetsAsianPut: K=[28.000, 13.000, 20.500], T=2
- MultiAssetsAsianCall: K=[46.585, 20.508, 34.681], T=2
Price: [ 3.88140851e-11 -3.04099242e-12  7.15822069e-12]
(SE) [0.12869814 0.10738034 0.15149177]
Delta [[ 0.34640692 -0.         -0.        ]
 [-0.          0.54395912 -0.        ]
 [-0.         -0.          0.46053043]]
------------------------------------------
MultiAssetsBestOfAsianCall: K=[46.585, 20.508, 34.681], R_required=0.045, T_entry=1, T_mature=2
Price: -4.145941486727937
(SE) 0.2592035656479525
Delta [[-0.25866813 -0.4312024  -0.34158998]]
------------------------------------------
MultiAssetsWorstOfDiscreteKIEuropeanPut: K=[24.080, 10.160, 15.515], H=[21.080, 7.160, 12.515], T_entry=1, T_mature=2
Price: 4.145941486727937
(SE) 0.20406566031517118
Delta [[-0.08854904 -0.08397295 -0.10873836]]
------------------------------------------


In [ ]:
structure = metric(np.array([25, 10, 20.5]), np.array([3, 3, 2]), R=0.045)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

In [ ]:
structure = metric(np.array([32, 14.0, 23.5]), np.array([5, 5, 5]), R=0.045)
print('----------------------------------------------')
for name, obj in structure.items():
    print(repr(obj))

In [ ]:
def err(params, alpha):
    K, D, R = params[:3], params[3:], 0.1
    x = -(100/adv.dot(K))*adv*np.exp(0.045*2)
    loss, structure, pct_adv, cap, deltas = structure_constructor(
        mv_hes, x, K, D, R, adv, 0.3, s_0, 100, alpha, paths)
    # print(structure)
    print(K, x, R)
    print('Loss:', loss)
    print('X/ADV:', pct_adv)
    print('Locked-in Capital:', cap)
    return np.sqrt(loss)